# Clase Práctica #10 (Compilación)

En esta clase estaremos implementando un **generador de parsers SLR(1)**. Nos apoyaremos en la API de lenguajes que llevamos implementando desde el semestre anterior.

Comencemos por importar la clase `Grammar`.

In [40]:
from cmp.pycompiler import Grammar

Como de costumbre trabajaremos sobre un lenguaje de expresiones aritméticas básicas. Sin embargo, esta vez podemos finalmente usar la gramática natural de expresiones puesto que, como estudiamos en conferencia, los parser SLR(1) a diferencia de los LL(1) no son susceptibles a la presencia de recursión izquierda ni de prefijos comunes. Esto posibilita que la asociatividad a la izquierda de muchos de los operadores pueda ser representado sin problema.

In [41]:
G = Grammar()
E = G.NonTerminal('E', True)
T,F = G.NonTerminals('T F')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) int')

E %= E + plus + T | T # | E + minus + T 
T %= T + star + F | F # | T + div + F
F %= num | opar + E + cpar

print(G)

Non-Terminals:
	E, T, F
Terminals:
	+, -, *, /, (, ), int
Productions:
	[E -> E + T, E -> T, T -> T * F, T -> F, F -> int, F -> ( E )]


## Items

Se provee una implementación de la clase `Item` para modelar los Items LR(0).

In [42]:
from cmp.pycompiler import Item

Los items se construyen a partir de una producción.

In [43]:
prod = E.productions[0]
print('Posibles items de',repr(prod))

for x in range(len(prod.Right)+1):
    item = Item(prod, x)
    print('---------------------------------')
    print('item:', item)
    print('item.pos:', item.pos)
    print('item.IsReduceItem:', item.IsReduceItem)
    print('item.NextSymbol:', item.NextSymbol)
    print('item.NextItem():', item.NextItem())

Posibles items de E -> E + T
---------------------------------
item: E -> .E+T, 
item.pos: 0
item.IsReduceItem: False
item.NextSymbol: E
item.NextItem(): E -> E.+T, 
---------------------------------
item: E -> E.+T, 
item.pos: 1
item.IsReduceItem: False
item.NextSymbol: +
item.NextItem(): E -> E+.T, 
---------------------------------
item: E -> E+.T, 
item.pos: 2
item.IsReduceItem: False
item.NextSymbol: T
item.NextItem(): E -> E+T., 
---------------------------------
item: E -> E+T., 
item.pos: 3
item.IsReduceItem: True
item.NextSymbol: None
item.NextItem(): None


## Autómata LR(0)

Usaremos la implementación por referencia de autómata. Recordemos que bajo esta representación, los estados están conectados por referencias y el autómata resulta de seleccionar uno de ellos como raíz y expandir el grafo a partir de él.

In [44]:
from cmp.automata import State, lr0_formatter

La API de gramáticas provee la función `AugmentedGrammar` que construye una nueva gramática a partir de aumentar otra. Los símbolos y producciones de la gramática original se mantienen con las mismas referencias. Si el símbolo distinguido de la gramática a aumentar nunca aparece en parte derecha se devuelve la propia gramática. Es posible forzar el aumento de la gramática al incluir el argumento `force=True` al llamar a la función.

In [45]:
GG = G.AugmentedGrammar()

assert len(GG.startSymbol.productions) == 1
start_production = GG.startSymbol.productions[0]
start_production

S' -> E

In [46]:
start_item = Item(start_production, 0)
start_item

S' -> .E, 

### Construcción del autómata LR(0)

Implementemos el algoritmo para construir la versión no determinista del autómata LR(0). Recordemos de conferencia que:
- Cada item representa un estado.
- El estado inicial es representado por el item $S' \to .S$
- Todos los estados son finales: _Todo prefijo de un prefijo viable es un prefijo viable_.
    - Una cadena no es un prefijo viable si el autómata se traba.
- Función de transición:
    - $(X \to \alpha . c \beta) \longrightarrow^{c} (X \to \alpha c . \beta)$, con $c \in V_T$
    - $(X \to \alpha . Y \beta) \longrightarrow^{Y} (X \to \alpha Y . \beta)$, con $Y \in V_N$
    - $(X \to \alpha . Y \beta) \longrightarrow^{\epsilon} (Y \to .\delta)$, con $Y \in V_N$

In [47]:
# NOTA: use symbol.Name al hacer las transiciones, no directamente symbol.
def build_LR0_automaton(G):
    assert len(G.startSymbol.productions) == 1, 'Grammar must be augmented'

    start_production = G.startSymbol.productions[0]
    start_item = Item(start_production, 0)

    automaton = State(start_item, True)

    pending = [ start_item ]
    visited = { start_item: automaton }

    while pending:
        
        current_item = pending.pop()
        if current_item.IsReduceItem:
            continue
        
        next_symbol = current_item.NextSymbol
        transitions = {}
        epsilon_transitions = set()
        
        # Your code here!!! (Decide which transitions to add)
        
        if current_item.NextSymbol.IsNonTerminal:
            for prod in next_symbol.productions:
                item = Item(prod, 0)
                if item not in visited:
                    visited[item] = State(item, True)
                    pending.append(item)
                # '' denota epsilon-transicion
                epsilon_transitions.add(visited[item])
            # print('pending :',pending)
            next_item = current_item.NextItem()
            state = State(next_item, True)
            transitions[str(next_symbol)] = state
            if next_item not in visited:
                visited[next_item] = state
                pending.append(next_item)
        
        else:
            item = current_item.NextItem()
            state = State(item, True)
            if item not in visited:
                visited[item] = state
                pending.append(item)
            transitions[str(next_symbol)] = state
        
        # print("current_state", visited[current_item])
        # print("transitions", transitions)
        # print('epsilon_trans', epsilon_transitions)

        # Your code here!!! (Add the decided transitions)
        for x in transitions:
            visited[current_item].add_transition(x, transitions[x])
        
        for x in epsilon_transitions:
            visited[current_item].epsilon_transitions.add(x)
    
    return automaton

Al correr el algoritmo debemos obtener la versión no determinista del autómata. Recordemos que este autómata reconoce el lenguaje de los prefijos viables de una gramática: cadenas que pueden ocurrir en la pila durante el parseo de una cadena válida.

In [48]:
automaton = build_LR0_automaton(GG)
print(automaton._repr_svg_())

assert automaton.recognize('E')
assert automaton.recognize('T*F')
assert automaton.recognize(['E', '+', 'int'])
assert not automaton.recognize('E*F')

automaton.set_formatter(lr0_formatter)
display(automaton._repr_svg_())

None


None

Para construir la versión determinista del autómata LR(0) simplemente aplicaremos el método `to_deterministic` que implementamos en clases anteriores. En conferencia estudiamos que es posible construir el autómata determinista directamente a partir de calcular la **Colección Canónica de Items LR(0)**. Esta variante queda propuesta a implementar como estudio individual.

In [49]:
automaton.to_deterministic(lr0_formatter)

(T -> .F, , F -> .int, , E -> .E+T, , S' -> .E, , F -> .(E), , E -> .T, , T -> .T*F, )

## Parsers Shift-Reduce

Un parser *shift-reduce* es un mecanismo de parsing que cuenta con las siguientes estructuras:

- Una pila de símbolos `S`.
- Una secuencia de terminales `T`.

> Denotamos el estado del parser como $\alpha|\omega$, con $S = \alpha$ y $T = \omega$.

Y las operaciones siguientes:

- **shift**: Si el parser se encuentra en un estado $\alpha | c \omega$, entonces tras aplicar una operación _shift_ pasa al estado $\alpha c | \omega$.
- **reduce**: Si el parser se encuentra en un estado $\alpha \beta | \omega$, y $X \rightarrow \beta$ es una producción, entonces tras aplicar una operación _reduce_ $T \rightarrow \beta$ pasa al estado $\alpha X | \omega$.

Podemos definir entonces el proceso de parsing como:

> Sea $S = \emptyset$ la pila inicial, $T = \omega \$$ la cadena a reconocer, y $E$ el símbolo inicial, un parser shift-reduce reconoce esta cadena si y solo si existe una secuencia de operaciones **shift** y **reduce** tal que tras aplicarlas se obtiene $S = E$ y $T = \$$.

Todos los algoritmos de parsing que estudiaremos en este semestre están basados en esta arquitectura. La diferencia entre ellos radica justamente en la forma en que deciden entre hacer _shift_ o _reduce_.

Para implementarlos, nos apoyaremos en una representación uniforme: tabla **Acción-Goto**, la cual sigue la siguiente estructura:


          ________ _______________________ ___________
         |________|_________ACTION________|___GOTO____|
         | Estado | +   *   (   )  int  $ | E   T   F |
         |--------|--- --- --- --- --- ---|--- --- ---|
         |   ...  |          ...          |    ...    |
         |________|_______________________|___________|

donde para todo $I_i$ estado de la Colección Canónica, $c \in V_T \cup \{ \$ \}$ y $X,Y \in V_N$.

- $ACTION[I_i, c] \in \{ `S_k`, `R_k`, `OK` \}$
- $GOTO[I_i, Y] \in \{ 1...N \}$


In [50]:
class ShiftReduceParser:
    SHIFT = 'SHIFT'
    REDUCE = 'REDUCE'
    OK = 'OK'
    
    def __init__(self, G, verbose=False):
        self.G = G
        self.verbose = verbose
        self.action = {}
        self.goto = {}
        self._build_parsing_table()
    
    def _build_parsing_table(self):
        raise NotImplementedError()

    def __call__(self, w, get_shift_reduce=False):
        stack = [ 0 ]
        cursor = 0
        output = []
        operations = []
        while True:
            state = stack[-1]
            lookahead = w[cursor]
            if self.verbose: print(stack, '<---||--->', w[cursor:])
            
            if(state,lookahead) not in self.action:
                print((state,lookahead))
                print(self.action)
                print("Error. Aborting...")
                return None
            
            print('state, lookahead', state, lookahead)
            print(self.action)
            if self.action[state, lookahead] == self.OK:
                action = self.OK
            else:
                action, tag = self.action[state, lookahead]
            print('action, tsg', action)
            if action==self.SHIFT:
                operations.append(self.SHIFT)
                stack+=[lookahead,tag]
                cursor+=1
            elif action==self.REDUCE:    
                operations.append(self.REDUCE)
                output.append(tag)
                print('tag', tag)
                head,body=tag    
                for symbol in reversed(body):
                    print('stack', stack)
                    stack.pop()    
                    
                    assert stack.pop()==symbol
                    state=stack[-1]
                    print(self.goto,'goto')
                    print('output', output)
                goto=self.goto[state,head]
                stack+=[head,goto]
            elif action==self.OK:
                stack.pop()
                assert stack.pop()==self.G.startSymbol
                assert len(stack)==1    
                return output if not get_shift_reduce else(output,operations)
            else:    
                raise Exception('Invalid action!!!')
            


### Cómo llena la tabla un parser SLR(1)?

- **Sea** $X \to \alpha .c \omega$ un item del estado $I_i$ y $Goto(I_i,c) = I_j$. **Entonces** $ACTION[I_i,c] = `S_j`$.

- **Sea** $X \to \alpha .$ un item del estado $I_i$ y $c \in FOLLOW(X)$. **Entonces** $ACTION[I_i,c] = `R_k`$ (producción `k` es $X \to \alpha$).

- **Sea** $I_i$ el estado que contiene el item $S' \to S.$ ($S'$ distinguido). **Entonces** $ACTION[I_i,\$] = `OK`$.

- **Sea** $X \to \alpha .Y \omega$ item del estado $I_i$ y $Goto(I_i,Y) = I_j$. **Entonces** $GOTO[I_i,Y] = j$.

In [51]:
from cmp.tools.parsing import compute_firsts, compute_follows

class SLR1Parser(ShiftReduceParser):

    def _build_parsing_table(self):
        G = self.G.AugmentedGrammar(True)
        firsts = compute_firsts(G)
        follows = compute_follows(G, firsts)
        # print('first:', firsts)
        # print('follow:', follows)
        automaton = build_LR0_automaton(G).to_deterministic()
        for i, node in enumerate(automaton):
            if self.verbose: print(i, '\t', '\n\t '.join(str(x) for x in node.state), '\n')
            node.idx = i
        for i,production in enumerate(G.Productions,1):
            production.j = i
        for node in automaton:
            idx = node.idx
            # print(node.transitions, 'kslkws')
            for state in node.state:
                # print('state :',state)
                item = state.state
                # Your code here!!!
                # print(item.NextItem().production)
                # print(item.NextItem().production.Left, 'cabecera')
                # print(item.NextSymbol, 'symbol')
                # print('type:', type(item.NextSymbol))
            
                if len(item.production.Right)==item.pos:
                        # print('elif')
                    if item.production.Left == G.startSymbol:
                        self.action[node.idx,G.EOF] = self.OK
                    else:
                        for terminal_follow in follows[item.production.Left]:
                            tag = None
                            for production in G.Productions:
                                if production == item.production:
                                    tag = production
                            self.action[node.idx,terminal_follow] = self.REDUCE,tag
                            
                elif item.production.Right[item.pos].IsTerminal:
                    # print('item:',item)
                    # print('que hay:',item.production.Right[item.pos])
                    # print(item.NextSymbol.Name)
                    # print(state.transitions,'trans with s')
                    # print(automaton.transitions)
                    # print(node.idx)
                    # print(node.transitions)
                    # print('symbol:',item.NextSymbol.Name)
                    next_node = node.get(item.NextSymbol.Name)
                    # automaton.
                    self.action[node.idx,item.NextSymbol] = self.SHIFT,next_node.idx 
                    
                else:
                    # print(node.idx)
                    # print(node.transitions)
                    # print('symbol:',item.NextSymbol.Name)
                    next_node = node.get(item.NextSymbol.Name)
                    # print(next_node.idx ,'id')
                    # print('next node:', next_node)
                    self.goto[node.idx,item.NextSymbol] = next_node.idx 
                    # print(self.goto)
               
        # print('actions:',self.action)
        # print('goto:', self.goto)
                    

    @staticmethod
    def _register(table, key, value):
        assert key not in table or table[key] == value, 'Shift-Reduce or Reduce-Reduce conflict!!!'
        table[key] = value

## Probando

Construyamos un parser SLR(1) para la gramática de las expresiones aritméticas.

In [52]:
parser = SLR1Parser(G, verbose=True)

0 	 T -> .F, 
	 S' -> .E, 
	 F -> .int, 
	 E -> .E+T, 
	 F -> .(E), 
	 E -> .T, 
	 T -> .T*F,  

1 	 S' -> E., 
	 E -> E.+T,  

2 	 E -> E+.T, 
	 T -> .F, 
	 F -> .int, 
	 F -> .(E), 
	 T -> .T*F,  

3 	 F -> (.E), 
	 T -> .F, 
	 F -> .int, 
	 E -> .E+T, 
	 F -> .(E), 
	 E -> .T, 
	 T -> .T*F,  

4 	 F -> (E.), 
	 E -> E.+T,  

5 	 F -> (E).,  

6 	 F -> int.,  

7 	 T -> F.,  

8 	 E -> T., 
	 T -> T.*F,  

9 	 T -> T*.F, 
	 F -> .int, 
	 F -> .(E),  

10 	 T -> T*F.,  

11 	 T -> T.*F, 
	 E -> E+T.,  



### Tablas

Para visualizar las tablas Action y Goto usaremos la clase `DataFrame` de `pandas`.

In [53]:
from pandas import DataFrame

def encode_value(value):
    try:
        action, tag = value
        if action == ShiftReduceParser.SHIFT:
            return 'S' + str(tag)
        elif action == ShiftReduceParser.REDUCE:
            return repr(tag)
        elif action ==  ShiftReduceParser.OK:
            return action
        else:
            return value
    except TypeError:
        return value

def table_to_dataframe(table):
    d = {}
    for (state, symbol), value in table.items():
        # print(state,'ss')
        # print(symbol,'sy')
        # print(value,'v')
        value = encode_value(value)
        try:
            d[state][symbol] = value
        except KeyError:
            d[state] = { symbol: value }

    return DataFrame.from_dict(d, orient='index', dtype=str)

Recordemos que:

- Debe haber a lo sumo una opción en cada celda.

- Deben aparecer todos los estados (salvo $I_0$) entre **ACTION** y **GOTO**.

- Deben aparecer todas las producciones entre los $R_k$ de **ACTION**.

In [54]:
display(table_to_dataframe(parser.action))
display(table_to_dataframe(parser.goto))

,int,(,$,+,),*
0,S6,S3,NaN,NaN,NaN,NaN
2,S6,S3,NaN,NaN,NaN,NaN
3,S6,S3,NaN,NaN,NaN,NaN
9,S6,S3,NaN,NaN,NaN,NaN
1,NaN,NaN,OK,S2,NaN,NaN
5,NaN,NaN,F -> ( E ),F -> ( E ),F -> ( E ),F -> ( E )
6,NaN,NaN,F -> int,F -> int,F -> int,F -> int
7,NaN,NaN,T -> F,T -> F,T -> F,T -> F
8,NaN,NaN,E -> T,E -> T,E -> T,S9
10,NaN,NaN,T -> T * F,T -> T * F,T -> T * F,T -> T * F


,F,E,T
0,7,1.0,8.0
2,7,NaN,11.0
3,7,4.0,8.0
9,10,NaN,NaN


### Parseando ...

Trabajemos sobre la cadena `int + int * int`. Si el parser está correctamente implementado deberíamos obtener una derivación extrema derecha en reverso que parta de la oración y llegue al símbolo distinguido.

In [55]:
derivation = parser([num, plus, num, star, num, G.EOF])

print(str(derivation))
assert str(derivation) == '[F -> int, T -> F, E -> T, F -> int, T -> F, F -> int, T -> T * F, E -> E + T]'

derivation

[0] <---||---> ['int', '+', 'int', '*', 'int', '$']
state, lookahead 0 int
{(0, 'int'): ('SHIFT', 6), (0, '('): ('SHIFT', 3), (1, '$'): 'OK', (1, '+'): ('SHIFT', 2), (2, 'int'): ('SHIFT', 6), (2, '('): ('SHIFT', 3), (3, 'int'): ('SHIFT', 6), (3, '('): ('SHIFT', 3), (4, ')'): ('SHIFT', 5), (4, '+'): ('SHIFT', 2), (5, '+'): ('REDUCE', F -> ( E )), (5, '*'): ('REDUCE', F -> ( E )), (5, ')'): ('REDUCE', F -> ( E )), (5, '$'): ('REDUCE', F -> ( E )), (6, '+'): ('REDUCE', F -> int), (6, '*'): ('REDUCE', F -> int), (6, ')'): ('REDUCE', F -> int), (6, '$'): ('REDUCE', F -> int), (7, '+'): ('REDUCE', T -> F), (7, '*'): ('REDUCE', T -> F), (7, ')'): ('REDUCE', T -> F), (7, '$'): ('REDUCE', T -> F), (8, '+'): ('REDUCE', E -> T), (8, '$'): ('REDUCE', E -> T), (8, ')'): ('REDUCE', E -> T), (8, '*'): ('SHIFT', 9), (9, 'int'): ('SHIFT', 6), (9, '('): ('SHIFT', 3), (10, '+'): ('REDUCE', T -> T * F), (10, '*'): ('REDUCE', T -> T * F), (10, ')'): ('REDUCE', T -> T * F), (10, '$'): ('REDUCE', T -> T * F)

[F -> int, T -> F, E -> T, F -> int, T -> F, F -> int, T -> T * F, E -> E + T]

## Propuestas

- Complete el pipeline de evaluación. Observe que esta vez la gramática asocia naturalmente a la izquierda, lo cual debe simplificar la implementación de las reglas semánticas.

- Construya directamente la versión determinista del autómata LR(0).

- Explore otras gramáticas.

In [56]:
TESTING = True
if TESTING:
    GG = Grammar()

    S = GG.NonTerminal('S', True)
    V,A,E = GG.NonTerminals('V A E')
    i, equal, obrac, cbrac, plus = GG.Terminals('i = [ ] +')

    S %= V + equal + E
    V %= i | A + obrac + E + cbrac
    A %= i
    E %= E + plus + V | V

    GG = GG.AugmentedGrammar(True)
    automaton = build_LR0_automaton(GG)
    display(automaton.set_formatter(lr0_formatter))
    display(automaton.to_deterministic(lr0_formatter))

S' -> .S, 

(A -> .i, , V -> .A[E], , S -> .V=E, , S' -> .S, , V -> .i, )